<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [28]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import lxml

and we will provide some helper functions for you to process web scraped HTML table


In [29]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [30]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [31]:
# use requests.get() method with the provided static_url
# assign the response to a object
response=requests.get(static_url).text

Create a `BeautifulSoup` object from the HTML `response`


In [32]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup=BeautifulSoup(response,'html.parser')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Print the page title to verify if the `BeautifulSoup` object was created properly


In [33]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [59]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
df_list = pd.read_html(static_url,flavor='html5lib')

In [60]:
import string
translator=str.maketrans('','',string.punctuation + " ")

for i in range(0,9):
    cols=df_list[i+2].columns
    fixed_cols=cols.str.upper().str.translate(translator)
    df_list[i+2].columns=fixed_cols

    

In [61]:
df = pd.concat(df_list[2:10])
df.reset_index(inplace=True)
translator=str.maketrans('','',string.punctuation + " ")
df['DATEANDTIMEUTC']=df['DATEANDTIMEUTC'].str.translate(translator)
df
tst_df = df[df['DATEANDTIMEUTC'].str[0].str.isdigit()]
tst_df=tst_df.loc[~tst_df['DATEANDTIMEUTC'].str.contains("planned")]
tst_df['DATEANDTIMEUTC']=pd.to_datetime(tst_df['DATEANDTIMEUTC'],format='%d%B%Y%H%M%S%f')
tst_df = tst_df.groupby('FLIGHTNO').filter(lambda x : len(x)==1 )
tst_df

,index,FLIGHTNO,DATEANDTIMEUTC,VERSIONBOOSTERB,LAUNCHSITE,PAYLOADC,PAYLOADMASS,ORBIT,CUSTOMER,LAUNCHOUTCOME,BOOSTERLANDING
0,0,1,2010-06-04 01:08:04.500,F9 v1.0[7]B0003.1[8],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure[9][10](parachute)
2,2,2,2010-12-08 15:43:01.300,F9 v1.0[7]B0004.1[8],"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),NaN,LEO (ISS),NASA (COTS) NRO,Success[9],Failure[9][14](parachute)
4,4,3,2012-05-22 07:44:01.700,F9 v1.0[7]B0005.1[8],"CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt
9,9,5,2013-03-01 01:05:01.000,F9 v1.0[7]B0007.1[8],"CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg (10,752 lb)",LEO (ISS),NASA (CRS),Success,No attempt
11,11,6,2013-09-29 16:00:03.000,F9 v1.1[7]B1003[8],"VAFB,SLC-4E",CASSIOPE[22][31],"500 kg (1,100 lb)",Polar orbit LEO,MDA,Success[30],Uncontrolled(ocean)[d]
...,...,...,...,...,...,...,...,...,...,...,...
211,42,99,2020-11-21 17:17:08.587,F9 B5B1063.1,"VAFB,SLC-4E",Sentinel-6 Michael Freilich (Jason-CS A),"1,192 kg (2,628 lb)",LEO,NASA / NOAA / ESA / EUMETSAT,Success,Success(ground pad)
213,44,100,2020-11-25 02:13:58.900,F9 B5 ♺B1049.7[590],"CCAFS,SLC-40",Starlink 15 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship)
215,46,101,2020-12-06 16:17:08.591,F9 B5 ♺B1058.4[592],"KSC,LC-39A",SpaceX CRS-21(Dragon C208.1),"2,972 kg (6,552 lb)",LEO (ISS),NASA (CRS),Success,Success(drone ship)
217,48,102,2020-12-13 17:30:00.596,F9 B5 ♺B1051.7,"CCSFS,SLC-40[597]",SXM-7,"7,000 kg (15,000 lb)",GTO,Sirius XM,Success,Success(drone ship)


In [62]:
pd.to_datetime(tst_df['DATEANDTIMEUTC'],'%d %B %Y')
tst_df

,index,FLIGHTNO,DATEANDTIMEUTC,VERSIONBOOSTERB,LAUNCHSITE,PAYLOADC,PAYLOADMASS,ORBIT,CUSTOMER,LAUNCHOUTCOME,BOOSTERLANDING
0,0,1,2010-06-04 01:08:04.500,F9 v1.0[7]B0003.1[8],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure[9][10](parachute)
2,2,2,2010-12-08 15:43:01.300,F9 v1.0[7]B0004.1[8],"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),NaN,LEO (ISS),NASA (COTS) NRO,Success[9],Failure[9][14](parachute)
4,4,3,2012-05-22 07:44:01.700,F9 v1.0[7]B0005.1[8],"CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt
9,9,5,2013-03-01 01:05:01.000,F9 v1.0[7]B0007.1[8],"CCAFS,SLC-40",SpaceX CRS-2[22](Dragon C104),"4,877 kg (10,752 lb)",LEO (ISS),NASA (CRS),Success,No attempt
11,11,6,2013-09-29 16:00:03.000,F9 v1.1[7]B1003[8],"VAFB,SLC-4E",CASSIOPE[22][31],"500 kg (1,100 lb)",Polar orbit LEO,MDA,Success[30],Uncontrolled(ocean)[d]
...,...,...,...,...,...,...,...,...,...,...,...
211,42,99,2020-11-21 17:17:08.587,F9 B5B1063.1,"VAFB,SLC-4E",Sentinel-6 Michael Freilich (Jason-CS A),"1,192 kg (2,628 lb)",LEO,NASA / NOAA / ESA / EUMETSAT,Success,Success(ground pad)
213,44,100,2020-11-25 02:13:58.900,F9 B5 ♺B1049.7[590],"CCAFS,SLC-40",Starlink 15 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship)
215,46,101,2020-12-06 16:17:08.591,F9 B5 ♺B1058.4[592],"KSC,LC-39A",SpaceX CRS-21(Dragon C208.1),"2,972 kg (6,552 lb)",LEO (ISS),NASA (CRS),Success,Success(drone ship)
217,48,102,2020-12-13 17:30:00.596,F9 B5 ♺B1051.7,"CCSFS,SLC-40[597]",SXM-7,"7,000 kg (15,000 lb)",GTO,Sirius XM,Success,Success(drone ship)


In [63]:
df_list = pd.concat(df_list[2:10])
df_list.reset_index(inplace=True)
df_list['DATEANDTIMEUTC']=df_list['DATEANDTIMEUTC'].str.replace('[^\w\s]','')

df_list=df_list[df_list['DATEANDTIMEUTC'].notnull()]
df_list=df_list[df_list['DATEANDTIMEUTC'].str[0].str.isdigit()]

/home/marlon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


Starting from the third table is our target table contains the actual launch records.


In [64]:
# Let's print the third table and check its content
df_list


,index,FLIGHTNO,DATEANDTIMEUTC,VERSIONBOOSTERB,LAUNCHSITE,PAYLOADC,PAYLOADMASS,ORBIT,CUSTOMER,LAUNCHOUTCOME,BOOSTERLANDING
0,0,1,4 June 20101845,F9 v1.0[7]B0003.1[8],"CCAFS,SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure[9][10](parachute)
2,2,2,8 December 2010154313,F9 v1.0[7]B0004.1[8],"CCAFS,SLC-40",Dragon demo flight C1(Dragon C101),NaN,LEO (ISS),NASA (COTS) NRO,Success[9],Failure[9][14](parachute)
4,4,3,22 May 2012074417,F9 v1.0[7]B0005.1[8],"CCAFS,SLC-40",Dragon demo flight C2+[18](Dragon C102),"525 kg (1,157 lb)[19]",LEO (ISS),NASA (COTS),Success[20],No attempt
6,6,4,8 October 2012003521,F9 v1.0[7]B0006.1[8],"CCAFS,SLC-40",SpaceX CRS-1[22](Dragon C103),"4,700 kg (10,400 lb)",LEO (ISS),NASA (CRS),Success,No attempt
7,7,4,8 October 2012003521,F9 v1.0[7]B0006.1[8],"CCAFS,SLC-40",Orbcomm-OG2[23],172 kg (379 lb)[24],LEO,Orbcomm,Partial failure[25],No attempt
...,...,...,...,...,...,...,...,...,...,...,...
211,42,99,21 November 2020171708587,F9 B5B1063.1,"VAFB,SLC-4E",Sentinel-6 Michael Freilich (Jason-CS A),"1,192 kg (2,628 lb)",LEO,NASA / NOAA / ESA / EUMETSAT,Success,Success(ground pad)
213,44,100,25 November 20200213589,F9 B5 ♺B1049.7[590],"CCAFS,SLC-40",Starlink 15 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success(drone ship)
215,46,101,6 December 2020161708591,F9 B5 ♺B1058.4[592],"KSC,LC-39A",SpaceX CRS-21(Dragon C208.1),"2,972 kg (6,552 lb)",LEO (ISS),NASA (CRS),Success,Success(drone ship)
217,48,102,13 December 2020173000596,F9 B5 ♺B1051.7,"CCSFS,SLC-40[597]",SXM-7,"7,000 kg (15,000 lb)",GTO,Sirius XM,Success,Success(drone ship)


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


Check the extracted column names


In [65]:
df_list.columns

Index(['index', 'FLIGHTNO', 'DATEANDTIMEUTC', 'VERSIONBOOSTERB', 'LAUNCHSITE',
       'PAYLOADC', 'PAYLOADMASS', 'ORBIT', 'CUSTOMER', 'LAUNCHOUTCOME',
       'BOOSTERLANDING'],
      dtype='object')

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [66]:
falcon9_data=df_list
falcon9_data.shape

(114, 11)

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
